In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from mnist_deep import deepnn
import numpy as np

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Normal mnist model

In [21]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None, 10])
y_conv, keep_prob = deepnn(x)
with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)
with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08
step 100, training accuracy 0.82
step 200, training accuracy 0.94
step 300, training accuracy 0.94
step 400, training accuracy 0.94
step 500, training accuracy 0.92
step 600, training accuracy 0.94
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.96
test accuracy 0.9615


In [40]:
class mnist_with_outlier:
    def __init__(self, mnist, no_outlier_numbers, outlier_numbers, outlier_ratio=0.3):
        self.mnist = mnist
        self.no_outlier_numbers = no_outlier_numbers
        self.outlier_numbers = outlier_numbers
        self.outlier_ratio = outlier_ratio
        self.train_images, self.train_labels = self.add_outlier(self.mnist.train.images, self.mnist.train.labels)
        self.test_images, self.test_labels = self.extract_test(self.mnist.test.images, self.mnist.test.labels)
        self._index_in_epoch = 0
        self._epochs_completed = 0
        self._num_examples = self.train_images.shape[0]
    
    def extract_test(self, images, labels):
        extracted_images, extracted_labels = self.extract_number(images, labels)
        extracted_labels = self.dense_to_one_hot(extracted_labels, len(self.no_outlier_numbers))
        return extracted_images, extracted_labels
    
    def extract_number(self, images, labels):
        mixed_images = np.array([])
        mixed_labels = np.array([])
        label_numbers = np.where(labels == 1)[1]
        for i in self.no_outlier_numbers:
            i_positions = np.where(label_numbers == i)[0].reshape(-1)
            if mixed_images.shape[0] == 0:
                mixed_images = images[i_positions]
            else:
                mixed_images = np.concatenate((mixed_images ,images[i_positions]))
            mixed_labels = np.concatenate((mixed_labels, np.array([i] * len(i_positions))))
        return mixed_images, mixed_labels
        
    def add_outlier(self, images, labels):
        mixed_images, mixed_labels = self.extract_number(images, labels)
        if self.outlier_ratio > 0:
            outlier_num = int(mixed_images.shape[0] * self.outlier_ratio)
            outlier_images = np.array([])
            label_numbers = np.where(labels == 1)[1]
            for j in self.outlier_numbers:
                j_positions = np.where(label_numbers == j)[0].reshape(-1)
                if outlier_images.shape[0] == 0:
                    outlier_images = images[j_positions]
                else:
                    outlier_images = np.concatenate((outlier_images ,images[j_positions]))

            outlier_images_samples = outlier_images[np.random.choice(len(outlier_images), outlier_num, replace=False)]
            outlier_labels_samples = np.random.choice(self.no_outlier_numbers, outlier_num, replace=True)

            mixed_images = np.concatenate((mixed_images, outlier_images_samples))
            mixed_labels = np.concatenate((mixed_labels, outlier_labels_samples))

            idx = np.random.permutation(len(mixed_labels))
            mixed_images, mixed_labels = mixed_images[idx], mixed_labels[idx]
        
        return mixed_images, self.dense_to_one_hot(mixed_labels, len(self.no_outlier_numbers))
    
    def dense_to_one_hot(self, labels, num_classes):
        labels_dense = np.array([self.no_outlier_numbers.index(i) for i in labels])
        num_labels = labels_dense.shape[0]
        index_offset = np.arange(num_labels) * num_classes
        labels_one_hot = np.zeros((num_labels, num_classes))
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
        return labels_one_hot
     
    def next_batch(self, batch_size, shuffle=True):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        # Go to the next epoch
        if start + batch_size > self._num_examples:
          # Finished epoch
          self._epochs_completed += 1
          # Get the rest examples in this epoch
          rest_num_examples = self._num_examples - start
          images_rest_part = self.train_images[start:self._num_examples]
          labels_rest_part = self.train_labels[start:self._num_examples]
          # Shuffle the data
          if shuffle:
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self.train_images = self.train_images[perm]
            self.train_labels = self.train_labels[perm]
          # Start next epoch
          start = 0
          self._index_in_epoch = batch_size - rest_num_examples
          end = self._index_in_epoch
          images_new_part = self.train_images[start:end]
          labels_new_part = self.train_labels[start:end]
          return np.concatenate((images_rest_part, images_new_part), axis=0) , np.concatenate((labels_rest_part, labels_new_part), axis=0)
        else:
          self._index_in_epoch += batch_size
          end = self._index_in_epoch
          return self.train_images[start:end], self.train_labels[start:end]

# Use cnn to classify number 6,7 while adding outliers 2 randomly labeled

In [37]:
mnist_outlier = mnist_with_outlier(mnist, [6,7], [2])
print(mnist_outlier.train_images.shape)
print(mnist_outlier.train_labels.shape)
print(mnist_outlier.test_images.shape)
print(mnist_outlier.test_labels.shape)

(14471, 784)
(14471, 2)
(1986, 784)
(1986, 2)


In [29]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None, 2])
y_conv, keep_prob = deepnn(x)
with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)
with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
      batch = mnist_outlier.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist_outlier.test_images, y: mnist_outlier.test_labels, keep_prob: 1.0}))

step 0, training accuracy 0.44
step 100, training accuracy 0.86
step 200, training accuracy 0.9
step 300, training accuracy 0.9
step 400, training accuracy 0.82
step 500, training accuracy 0.92
step 600, training accuracy 0.88
step 700, training accuracy 0.84
step 800, training accuracy 0.88
step 900, training accuracy 0.88
test accuracy 0.998993


In [38]:
mnist_simple = mnist_with_outlier(mnist, [6,7], [], outlier_ratio=0)
print(mnist_simple.train_images.shape)
print(mnist_simple.train_labels.shape)
print(mnist_simple.test_images.shape)
print(mnist_simple.test_labels.shape)

(11132, 784)
(11132, 2)
(1986, 784)
(1986, 2)


# Use cnn to classify number 6,7 while not adding outliers 

In [36]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None, 2])
y_conv, keep_prob = deepnn(x)
with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)
with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
      batch = mnist_simple.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist_simple.test_images, y: mnist_simple.test_labels, keep_prob: 1.0}))

step 0, training accuracy 0
step 100, training accuracy 1
step 200, training accuracy 1
step 300, training accuracy 1
step 400, training accuracy 0.98
step 500, training accuracy 1
step 600, training accuracy 1
step 700, training accuracy 1
step 800, training accuracy 1
step 900, training accuracy 1
test accuracy 0.998993


In [41]:
mnist_outlier_all = mnist_with_outlier(mnist, [6,7], [0,1,2,3,4,5,8,9], outlier_ratio=0.3)
print(mnist_outlier_all.train_images.shape)
print(mnist_outlier_all.train_labels.shape)
print(mnist_outlier_all.test_images.shape)
print(mnist_outlier_all.test_labels.shape)

(14471, 784)
(14471, 2)
(1986, 784)
(1986, 2)


# Use cnn to classify number 6,7 while adding other numbers as outliers randomly labeled

In [42]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None, 2])
y_conv, keep_prob = deepnn(x)
with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)
with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
      batch = mnist_outlier_all.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist_outlier_all.test_images, y: mnist_outlier_all.test_labels, keep_prob: 1.0}))

step 0, training accuracy 0.56
step 100, training accuracy 0.88
step 200, training accuracy 0.88
step 300, training accuracy 0.82
step 400, training accuracy 0.8
step 500, training accuracy 0.9
step 600, training accuracy 0.9
step 700, training accuracy 0.86
step 800, training accuracy 0.84
step 900, training accuracy 0.9
test accuracy 0.998489
